In [1]:
import datetime
import requests


def gen_secid(symbol):
    exchange = symbol[0:2]
    code = symbol[2:]
    if exchange == "SH":
        return f"1.{code}"
    elif exchange == "ZS":
        f3 = code[0:3]
        if f3 == "000":
            return f"1.{code}"

    return f"0.{code}"


# 不复权
def cn_chartbar(symbol, start=None, end=None):
    if not start:
        start = "20100101"
    else:
        start = start.replace("-", "")
    if not end:
        end = datetime.datetime.now().strftime("%Y%m%d")
    else:
        end = end.replace("-", "")
    secid = gen_secid(symbol)
    # 不复权 : 0; 前复权 : 1; 后复权 : 2
    url = f'https://push2his.eastmoney.com/api/qt/stock/kline/get?secid={secid}&fields1=f1%2Cf2%2Cf3%2Cf4%2Cf5' \
          f'&fields2=f51%2Cf52%2Cf53%2Cf54%2Cf55%2Cf56%2Cf57%2Cf58%2Cf59%2Cf60%2Cf61&klt=101' \
          f'&fqt=0&beg={start}&end={end}'
    req = requests.get(url, timeout=30)  # request比pandas加载快
    json = req.json()
    return json

In [41]:
symb = "SZ000001"
json = cn_chartbar(symb, "2021-01-01")

In [43]:
import pandas as pd
klines = json['data']['klines']
dt = [s.split(',') for s in klines]
ori_labels = ['date','open','close','high','low','volume','turnover','amplitude','chg_pct','chg','turnover_rate']
df = pd.DataFrame.from_records(dt, columns=ori_labels, index='date')
df['volume'] = df.volume.map(lambda x: f'{x}100')
labels = ['open','high','low','close','volume','turnover','amplitude','chg_pct','chg','turnover_rate']
df = df[labels] # 调整一下column顺序
df.to_csv("/Users/max/sz000001.csv")
# klines.insert(0,'date,open,close,high,low,volume,turnover,amplitude,chg_pct,chg,turnover_rate')
# for s in klines:
#         print(s)

# dt = [s.split(',') for s in klines]
# dt
# df = pd.DataFrame(klines, columns=['date','open','close','high','low','volume','turnover','amplitude','chg_pct','chg','turnover_rate'])
# df

In [ ]:
# dt['data'] #['klines']
# sh = cn_chartbar('SH000001')
# sh  #7775
# sz = cn_chartbar('SZ399001')
# sz    #7728
cy = cn_chartbar('SZ399006')
cy    #3007